In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pylab as plt

import pandas as pd
import numpy as np

In [236]:
#Load staff data
file_name = 'West_Way_Accreditation/20161207 FY16 West Way Accreditation Report.xlsx'

xl_file = pd.ExcelFile(file_name)
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

#Save DF for each course
df_SC = dfs['SC Report']
df_T = dfs['T Report']
df_SA = dfs['SA Report']
df_MT = dfs['MT Report']

In [237]:
#Load target data
targets = 'West_Way_Accreditation/FY16 West Way Dealer Accreditation Targets.xlsx'

xl_file = pd.ExcelFile(targets)
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

#Save DF for each course
df_targets = dfs['FY16 Targets']
df_targets.fillna(0,inplace=True)

In [238]:
#Adjusting the headers 
df_SC.columns = df_SC.iloc[[4]].values[0]
df_SC.drop(df_SC.index[0:5],inplace=True)

#Adjusting the headers 
df_T.columns = df_T.iloc[[4]].values[0]
df_T.drop(df_T.index[0:5],inplace=True)

#Adjusting the headers 
df_SA.columns = df_SA.iloc[[4]].values[0]
df_SA.drop(df_SA.index[0:5],inplace=True)

#Adjusting the headers 
df_MT.columns = df_MT.iloc[[4]].values[0]
df_MT.drop(df_MT.index[0:5],inplace=True)

In [240]:
#Drop common surplus headers
surplus_headers_SC = ['Current Accreditation Status', 'Completed/Booked all F2F criteria ',
           'Completed/Booked all Live criteria ', 'SC PDP ','Initial e-Learning ',
           'Annual Face to Face ', 'Annual VCT', 'Annual e-Learning ']

surplus_headers_T = ['Current Accreditation Status', 'Completed/Booked F2F criteria ',
                    'Completed/Booked all live criteria', 'Induction', 'e-Learning ',
                    'Annual e-Learning']

surplus_headers_SA = ['Current Accreditation Status', 'Completed/Booked F2F criteria ',
           'Completed/Booked all live criteria ', 'SA PDP ','e-Learning ',
           'Annual Face to Face ', 'Annual e-Learning ']

surplus_headers_MT = ['Current Accreditation Status', 'Completed/Booked all F2F criteria ',
           'Completed/Booked all Live criteria ', 'MT Assessment', 'e-Learning ',
           'Annual Update ', 'Annual e-Learning']

df_SC.drop(surplus_headers_SC,1,inplace=True)
df_T.drop(surplus_headers_T,1,inplace=True)
df_SA.drop(surplus_headers_SA,1,inplace=True)
df_MT.drop(surplus_headers_MT,1,inplace=True)

In [211]:
def aggregate_course(df):
    '''
    This function cleans up the data for a given accreditation criteria.
    Valuing completed coures as 1 and booked courses and 0.5. 
    Each course is weighted by the total number for that criteria and summed to give a fraction.
    The DataFrame is then update.
    '''
    #replace categoricals with numeric values
    df.replace('Complete',1.0,inplace=True)
    df.replace('Booked',0.5,inplace=True)
    df.fillna(0,inplace=True)
        
    #wieghting
    total_course_fraction = []
    l = len(df)
    
    for i in range(l):
        course_completion = df.iloc[[i]].values[0][5:]
        total_courses = len(course_completion)
        course_fraction = sum(map(lambda x: x/total_courses, course_completion))
        total_course_fraction.append(course_fraction)

    #Update Dataframe
    df['Total fraction'] = pd.Series(total_course_fraction, index=df.index)
    
    return df


In [243]:
#df_SC = aggregate_course(df_SC)
#df_T = aggregate_course(df_T)
#df_SA = aggregate_course(df_SA)
df_MT = aggregate_course(df_MT)

TypeError: unsupported operand type(s) for /: 'unicode' and 'int'

In [ ]:
df_MT


In [213]:
#Deffine a list of dealers and filter targets
dealers = []
results = {}

for i in df_SC['Code '].unique():
    dealers.append(i)
    
df_targets[df_targets['Code'].isin(dealers)]

for i in dealers:
    dg = df_SC.groupby('Code ').get_group(i)
    mean = round(dg['Total fraction'].mean(),4)
    results[i]=mean
    

In [214]:
results

{1525: 0.6531,
 1533: 0.2891,
 1535: 0.7344,
 1633: 0.5491,
 1650: 0.596,
 1651: 0.9665,
 1655: 0.4434,
 1692: 0.3398,
 1693: 0.582,
 1694: 0.4978,
 1708: 0.3469,
 1877: 0.7054,
 1878: 0.4643,
 1883: 0.3698}

In [206]:
df_SC.groupby('Code ').get_group(1877)

,Username,First Name,Last Name,Code,Dealership,N-GAGE Sales Consultant TNA,Nissan Sales 1,Nissan Sales 2,Added Value Sales,VCT: The Nissan Brand,...,NSSW e-Learning,Sales NSSW: Customer Quality in Action,Nissan Accessories and Personalisation,Nissan Warranty for Sales,Quality: Approaching the Customer,Quality: Handover,Quality: Mindfulness at Work,All New Micra Part 1,FY16 Sales Consultant Knowledge Retention Assessment (coming soon),Total fraction
12,230413,Wojciech,Swiatkowski,1877,1877 West Way Hanwell,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.780533
45,230410,Arya,Ali-Kamal,1877,1877 West Way Hanwell,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.905418
53,231192,Derek,Rose,1877,1877 West Way Hanwell,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.593205
54,232048,Tanasha,Julien,1877,1877 West Way Hanwell,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.499541
63,230409,Akhtar,Ali,1877,1877 West Way Hanwell,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.749311
70,228624,Clive,Millen,1877,1877 West Way Hanwell,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.593205
75,226290,Alan,Cooper,1877,1877 West Way Hanwell,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.811754


In [117]:
df_SC.columns

Index([                     228078,                    u'Roger',
                        u'Barrett',                        1650,
       u'1650 West Way Birmingham',      u'Accredited for FY16',
                            u'Yes',                       u'No',
                      u'Completed',                u'Completed',
                      u'Completed',                u'Completed',
                    u'In Progress',                 u'Complete',
                       u'Complete',                 u'Complete',
                       u'Complete',                 u'Complete',
                       u'Complete',                 u'Complete',
                       u'Complete',                 u'Complete',
                       u'Complete',                 u'Complete',
                       u'Complete',                 u'Complete',
                       u'Complete',                 u'Complete',
                       u'Complete',                 u'Complete',
                       u'

In [65]:
for i in df_SC.columns:
    print i

Username
First Name
Last Name
Code 
Dealership 
Current Accreditation Status
Completed/Booked all F2F criteria 
Completed/Booked all Live criteria 
SC PDP 
Initial e-Learning 
Annual Face to Face 
Annual VCT
Annual e-Learning 
N-GAGE Sales Consultant TNA
Nissan Sales 1 
Nissan Sales 2
Added Value Sales 
VCT: The Nissan Brand 
VCT: Market Awareness
VCT: Selling Nissan Accessories, Personalisation and Warranty
You+Nissan Customer Promise 
MICRA
NOTE
QASHQAI
NP300 NAVARA
MY16 LEAF 
X-TRAIL 
PULSAR
JUKE
e-NV200
NV200 
NV400
Nissan Technologies
Nissan Sales 3
Qashqai Product Update VCT
VCT: Delivering the Customer Promise
NSSW e-Learning
Sales NSSW: Customer Quality in Action
Nissan Accessories and Personalisation
Nissan Warranty for Sales
Quality: Approaching the Customer
Quality: Handover
Quality: Mindfulness at Work
All New Micra Part 1 
FY16 Sales Consultant Knowledge Retention Assessment (coming soon)


In [67]:
df_SC.head()

,Username,First Name,Last Name,Code,Dealership,Current Accreditation Status,Completed/Booked all F2F criteria,Completed/Booked all Live criteria,SC PDP,Initial e-Learning,...,VCT: Delivering the Customer Promise,NSSW e-Learning,Sales NSSW: Customer Quality in Action,Nissan Accessories and Personalisation,Nissan Warranty for Sales,Quality: Approaching the Customer,Quality: Handover,Quality: Mindfulness at Work,All New Micra Part 1,FY16 Sales Consultant Knowledge Retention Assessment (coming soon)
5,228112,Stephen,Duffy,1525,1525 West Way Basingstoke,NaN,Yes,No,Completed,In Progress,...,Complete,Complete,0,0,0,0,0,0,Complete,NaN
6,229504,Russell,Ord,1692,1692 West Way Stockport,NaN,NaN,No,In Progress,Completed,...,0,0,0,0,0,0,0,0,Complete,NaN
7,222462,John,Packwood,1655,1655 West Way Wolverhampton,NaN,Yes,No,Completed,Completed,...,0,0,0,0,0,0,0,0,Complete,NaN
8,231377,Adrian,Coleman,1633,1633 West Way Coventry,NaN,NaN,No,In Progress,In Progress,...,Complete,Complete,0,0,0,0,0,0,Complete,NaN
9,228808,Jonathon,Boswell,1708,1708 West Way Manchester,NaN,NaN,No,In Progress,In Progress,...,Complete,0,0,0,0,0,0,0,Complete,NaN
